<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:#4361EE"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Diplomado en Inteligencia Artificial y Aprendizaje Profundo</center></span>

# <span style="color:green"><center>Pytorch-lightning</center></span>

<figure>
<center>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Alejandria/main/Pytorch/Imagenes/Pytorch_ligthning_logo.png" width="600" height="200" align="center" /> 
</center>   
</figure>


## <span style="color:#4361EE">Introducción</span> 

Tomado de [Wikipedia](https://en.wikipedia.org/wiki/PyTorch_Lightning)

PyTorch Lightning es una biblioteca Python de código abierto que proporciona una interfaz de alto nivel para PyTorch , un marco de aprendizaje profundo popular. Es un marco liviano y de alto rendimiento que organiza el código PyTorch para desvincular la investigación de la ingeniería, lo que hace que los experimentos de aprendizaje profundo sean más fáciles de leer y reproducir. Está diseñada para crear modelos de aprendizaje profundo escalables que pueden ejecutarse fácilmente en hardware distribuido y mantener los modelos independientes del hardware.

En 2019, Lightning fue adoptado por NeurIPS Reproducibility Challenge como estándar para enviar el código PyTorch a la conferencia.

## <span style="color:#4361EE">Entrenamiento de un modelo con Pythorch-lightning</span> 

La siguiente imagen representa los diferentes objetos requeridos para entrenar un modelo supervisado de redes neuronales. En general cada uno de estos objetos está presente en cualquier marco de trabajo (framework) para el entrenamiento de una red neuronal. Estos componentes son:

<figure>
<img src="https://raw.githubusercontent.com/AprendizajeProfundo/Alejandria/main/Pytorch/Imagenes/trainer.png" width="800" heigh="600" valign="left" />   
</figure>

Fuente: Alvaro Montenegro

### <span style="color:#4CC9F0">Datos: entrenamiento, validación, prueba</span>

Para entrenar un modelo supervisado necesitamos datos. Por lo general los datos se separan en dos o tres grupos que llamamos:

* `datos de entrenamiento`. Usualmente 70% u 80% del total de datos, elegido al azar y que son los datos que verá el optimizador en el proceso de entrenamiento; 
* `datos de validación`. Usualmente 10% 0 20%. Son datos usados durante el proceso de entrenamiento para validación en línea del modelo mientras es entrenado. Usualmente se usan con la función de pérdida y con las métricas definidas en cada caso;
* `datos de prueba`. Son datos externos al entrenamiento. Usualmente 10% del total de dato y se usan una vez ha terminado del entrenamiento para evaluar si el modelo generaliza adecuadamente, es decir si tiene el poder productivo que se espera.


### <span style="color:#4CC9F0">Modelo: la red neuronal</span>

Es la red neuronal que se ha diseñado para resolver el problema que se tiene entre manos. Usualmente construimos una clase en la cual de defines los componentes que tendrá la red y que serás conjuntos de capas separadas de manera lógica. En Pytorch el modelo se implementa con el método `foreward`. En este método se determina exactamente cómo funciona la red neuronal. Es el método de cálculo de la red neuronal.

### <span style="color:#4CC9F0">Entrenador: pérdida optimizador, métricas</span>

Es la red neuronal que se ha diseñado para resolver el problema que se tiene entre manos. Usualmente construimos una clase en la cual de defines los componentes que tendrá la red y que serás conjuntos de capas separadas de manera lógica. En Pytorch el modelo se implementa con el método `foreward`. En este método se determina exactamente cómo funciona la red neuronal. Es el método de cálculo de la red neuronal. Es el objeto que se encarga de hacer el entrenamiento de la red. El entrenador (trainer) usualmente tiene dos componentes esenciales:

* `paso de optimización` en el cual se define lo que se debe hace en cada paso del proceso de optimización;
* `ciclo de entrenamiento` que define como ocurrirá tos el entrenamiento: `número de épocas`,  `número de pasos de optimización por época`, `criterios de parada`,  lo que escribirá en el flujo de datos para seguimiento y evaluación (`writer`), y otras cosas.


## <span style="color:#4361EE">Instalar Pytorch-lightning</span> 

En consola ejecute el siguiente comando. 

In [3]:
#conda install -c conda-forge pytorch-lightning

## <span style="color:#4361EE">Ejemplo de una plantilla para usar PyTorch Lightning</span> 

Como ejemplo vamos a implementar un clasificador para los datos de MNIST, usando Lightning. Esta es una plantilla bastante general y está basada en la plantilla desarrollada por los autores de PyTorch Lightning. Puede revisar el código original en [Github](https://github.com/Lightning-AI/deep-learning-project-template/blob/master/project/lit_image_classifier.py).

### <span style="color:#4CC9F0">Importa módulos</span>

In [4]:
import torch
import pytorch_lightning as pl
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split

from torchvision.datasets.mnist import MNIST
from torchvision import transforms

import torchmetrics # métricas

from pytorch_lightning import seed_everything # reproducibilidad

### <span style="color:#4CC9F0">Clase Backbone: Nuestro modelo neuronal</span>

Definimos nuestra red neuronal con la clase *Backbone* que deriva de  la clase `torch.nn.Module`. Al hacer esto, se gana toda la implementacion disponible en la clase Definimos nuestra red neuronal con la clase *Backbone* que deriva de  la clase *torch.nn.Module*.

In [5]:
# modelo

class Backbone(torch.nn.Module):
    def __init__(self, hidden_dim=128):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, hidden_dim)
        self.l2 = torch.nn.Linear(hidden_dim, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.l1(x))
        x = torch.relu(self.l2(x))
        return x

### <span style="color:#4CC9F0">Clase  LitClassifier Nuestra clase Lightning</span>

Esta clase heredada de `pl.LightningModule`  nos permite implementar los métodos que usará el entrenador para entrenar nuestra red. Es necesario especificar:

* el método de cálculo de la red (`forward`). En realidad, podemos definir la red dentro de esta clase, pero para desacoplar la rede y hacerlo esta clase más genérica, mejor definimos el modelo por fuera. de esta clase pytorch-lightning;
* el paso de entrenamiento, en el cual adicionalmente escribimos en el log para Tensorboard;
* el paso de validación, que también escribe el en log para Tensorboard;
* el paso de prueba, que también escribe el en log para Tensorboard; 
* se configura el optimizador;
* las métricas de evaluación se definen en el constructor. Estas se van acumulando en cada paso de datos por el optimizador (trainig_step) y deben ser reinicializadas al final de cada época. De esto último se encarga PyTorch-Lightning de manera automática.


In [6]:
class LitClassifier(pl.LightningModule):
    def __init__(self, backbone, learning_rate=1e-3):
        super().__init__()
        # modelo
        self.backbone = backbone
        # métricas
        self.train_acc = torchmetrics.Accuracy()
        self.valid_acc = torchmetrics.Accuracy()
        self.test_acc = torchmetrics.Accuracy()
        # rata de aprendizaje
        self.lr = learning_rate

    def forward(self, x):
        # use forward para inferencia/predicciones
        embedding = self.backbone(x)
        return embedding

    def training_step(self, batch, batch_idx):
        # recibe batch de datos de entrenamiento
        x, y = batch
        # calcula el modelo(la red) con estos datos
        y_hat = self.backbone(x)
        # calcula la pérdida para estos datos
        loss = F.cross_entropy(y_hat, y)
        # sube la pérdida de entrenamiento al log 
        self.log('train_loss', loss, on_epoch=True)
        # actualiza las métricas para los datos de entrenamiento
        self.train_acc(y_hat, y)
        # sube las métricas de los datos de entrenamiento al log 
        self.log('train_acc', self.train_acc, on_step=True, on_epoch=False)
        # retorna la pérdida
        return loss

    def validation_step(self, batch, batch_idx):
        # recibe batch de datos de valicación
        x, y = batch
        # calcula el modelo(la red) con estos datos
        y_hat = self.backbone(x)
        # calcula la pérdida para estos datos
        loss = F.cross_entropy(y_hat, y)
        # actualiza las métricas para los datos de validación
        self.valid_acc(y_hat, y)
        # sube las métricas de los datos de validación al log 
        self.log('valid_acc', self.valid_acc, on_step=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        # recibe batch de datos de prueba
        x, y = batch
         # calcula el modelo(la red) con estos datos
        y_hat = self.backbone(x)
        # calcula la pérdida para estos datos
        loss = F.cross_entropy(y_hat, y)
        # sube la pérdida de prueba al log
        self.log('test_loss', loss)
         # actualiza las métricas para los datos de validación
        self.test_acc(y_hat, y)
        # sube las métricas de los datos de test al log 
        self.log('test_acc', self.test_acc, on_step=True, on_epoch=True)


        
    def training_epoch_end(self, outs):
        # define aqui las acciones al terminar una época en entrenamiento
        pass
    
    def validation_epoch_endd(self, outs):
        # define aqui las acciones al terminar una época en validación
        pass
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

 

### <span style="color:#4CC9F0">Datos: Datasets y Dataloaders</span>

In [7]:
# Tamaño de los lotes de datos para el entrenamiento
batch_size = 32
# número de trabajadores para distribuir el trabajo de los dataloaders
num_workers=4
# Cuando mezclar los bloques de datos
shuffle = True

# creación de datasets desde Pytorch
dataset = MNIST('', train=True, download=True, transform=transforms.ToTensor())
mnist_test = MNIST('', train=False, download=True, transform=transforms.ToTensor())
mnist_train, mnist_val = random_split(dataset, [55000, 5000])

# creación de los dataloaders
train_loader = DataLoader(mnist_train, batch_size=batch_size, num_workers=num_workers, shuffle=shuffle)
val_loader = DataLoader(mnist_val, batch_size=batch_size,num_workers=num_workers)
test_loader = DataLoader(mnist_test, batch_size=batch_size,num_workers=num_workers)

2.6%

100.0%


Extracting MNIST\raw\train-images-idx3-ubyte.gz to MNIST\raw



100.0%

Extracting MNIST\raw\train-labels-idx1-ubyte.gz to MNIST\raw




7.9%

100.0%


Extracting MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST\raw



100.0%

Extracting MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST\raw



### <span style="color:#4CC9F0">Entrenador</span>

Instancia un entrenador que será una instancia de `Trainer`.

#### Reproducibilidad

Para garantizar la reproducibilidad total de una ejecución a otra, debemos configurar semillas para generadores pseudoaleatorios y establecer una bandera determinista en *Trainer*. Usaremos *seed_everything*.

Aquí *Workers=True* en *seed_everything()*, Lightning obtiene semillas únicas en todos los procesos y trabajadores del cargador de datos para generadores de números aleatorios *torch, numpy y stdlib*. Cuando está encendido, asegura que, p. los aumentos de datos no se repiten entre los trabajadores.

También deberíamos notar: determinista=Verdadero en entrenador.

Sin embargo, si solo planea establecer una semilla aleatoria para python, numpy, pytorch. y no usas pytorch lightning para entrenar. seed_everything() es de hecho.

Porque seed_everything() se implementa de la siguiente manera:

In [8]:
'''
os.environ["PL_GLOBAL_SEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
'''

'\nos.environ["PL_GLOBAL_SEED"] = str(seed)\nrandom.seed(seed)\nnp.random.seed(seed)\ntorch.manual_seed(seed)\ntorch.cuda.manual_seed_all(seed)\n'

#### Aceleradores de hardware

Con PyTorch Lightning es posible actualmente utilizar los siguientes tipos de aceleradores de hardware

* GPU
* TPU
* IPU
* HPU


Tenga en cuenta que Pytorch no trabaja bien con múltiples cpu's y gpu's en entrenamiento. Nuestro consejo es:

* Use múltiples procesos (workers) en los dataloaders.
* Use múltiples gpu's en el Trainer.


#### Instanciando el entrenador

In [9]:
#trainer = pl.Trainer(gpus=4, precisión=16, limit_train_batches=0.5)
max_epochs = 10
acelerator = 'cpu'

# reproducibilidad
seed_everything(42, workers=True)
# Al escribir la línea anterior es necesario escribir. 
# Caso contrario deterministic=False, que se tiene por defecto
deterministic=True
# Cuántas gpus usar
gpus = 0 # depende de sus recursos

trainer = pl.Trainer(accelerator=acelerator, max_epochs= max_epochs, 
                     deterministic=deterministic, gpus=gpus)

Global seed set to 42
c:\Users\User\anaconda3\envs\work\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### <span style="color:#4CC9F0">Entrenamiento</span>

In [10]:
hidden_dim = 128

model = LitClassifier(Backbone(hidden_dim=hidden_dim))
trainer.fit(model, train_loader, val_loader)

Missing logger folder: c:\Users\User\OneDrive\Documentos\GitHub\Libro_Fundamentos_Programacion\Pytorch\Cuadernos\lightning_logs

  | Name      | Type     | Params
---------------------------------------
0 | backbone  | Backbone | 101 K 
1 | train_acc | Accuracy | 0     
2 | valid_acc | Accuracy | 0     
3 | test_acc  | Accuracy | 0     
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 1876/1876 [01:13<00:00, 25.51it/s, loss=0.488, v_num=0]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 1876/1876 [01:13<00:00, 25.50it/s, loss=0.488, v_num=0]


### <span style="color:#4CC9F0">Prueba</span>

In [11]:
result = trainer.test(dataloaders=test_loader)
print(result)

c:\Users\User\anaconda3\envs\work\lib\site-packages\pytorch_lightning\trainer\trainer.py:1386: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at c:\Users\User\OneDrive\Documentos\GitHub\Libro_Fundamentos_Programacion\Pytorch\Cuadernos\lightning_logs\version_0\checkpoints\epoch=9-step=17190.ckpt
Loaded model weights from checkpoint at c:\Users\User\OneDrive\Documentos\GitHub\Libro_Fundamentos_Programacion\Pytorch\Cuadernos\lightning_logs\version_0\checkpoints\epoch=9-step=17190.ckpt


Testing DataLoader 0: 100%|██████████| 313/313 [00:03<00:00, 85.81it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_acc_epoch         0.7681999802589417
        test_loss           0.5698840022087097
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[{'test_loss': 0.5698840022087097, 'test_acc_epoch': 0.7681999802589417}]


## <span style="color:#4361EE">Lanzar Tensorboard para examinar los resultados</span>

Terminamos esta lección ejecutando Tensorboard, la poderosa herramienta desarrollada por Google para visualizar los resultados del entrenamiento. Tensorboard tiene muchas características interesantes. Revise las lecciones de Tonsorboard para TensorFlow y PyTorch.

In [12]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/